In [2]:
# BEGIN FLAGS_PY
import os
import time
import sys

import requests  # <1>

POP20_CC = ('CN IN US ID BR PK NG BD RU JP '
            'MX PH VN ET EG DE IR TR CD FR').split()  # <2>

BASE_URL = 'http://flupy.org/data/flags'  # <3>

DEST_DIR = 'downloads/'  # <4>


def save_flag(img, filename):  # <5>
    path = os.path.join(DEST_DIR, filename)
    with open(path, 'wb') as fp:
        fp.write(img)


def get_flag(cc):  # <6>
    url = '{}/{cc}/{cc}.gif'.format(BASE_URL, cc=cc.lower())
    resp = requests.get(url)
    return resp.content


def show(text):  # <7>
    print(text, end=' ')
    sys.stdout.flush()


def download_many(cc_list):  # <8>
    for cc in sorted(cc_list):  # <9>
        image = get_flag(cc)
        show(cc)
        save_flag(image, cc.lower() + '.gif')

    return len(cc_list)


def main(download_many):  # <10>
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))


if __name__ == '__main__':
    main(download_many)  # <11>
# END FLAGS_PY


BD BR CD CN DE EG ET FR ID IN IR JP MX NG PH PK RU TR US VN 
20 flags downloaded in 81.90s


In [5]:
from concurrent import futures

MAX_WORKERS = 20

def downloud_one(cc):
    image = get_flag(cc)
    show(cc)
    save_flag(image, cc.lower() + ".gif")
    return cc

# ++++++++++++++++++++++++++++++++++++++++
def download_many_cc(cc_list):
    workers = min(MAX_WORKERS, len(cc_list))
    with futures.ThreadPoolExecutor(workers) as executor :
        res = executor.map(downloud_one, sorted(cc_list))
    
    return len(list(res))
# ----------------------------------------

def main(download_many):  # <10>
    t0 = time.time()
    count = download_many(POP20_CC)
    elapsed = time.time() - t0
    msg = '\n{} flags downloaded in {:.2f}s'
    print(msg.format(count, elapsed))


if __name__ == '__main__':
    main(download_many_cc)  # <11>

EG JP CNDE  CD RU ID PK NG ET TR IR VN US MX BR BD IN FR PH 
20 flags downloaded in 2.52s


做个简单的比喻：进程（process）=火车，线程（thread）=车厢

- 线程在进程下行进（单纯的车厢无法运行）
- 一个进程可以包含多个线程（一辆火车可以有多个车厢）
- 不同进程间数据很难共享（一辆火车上的乘客很难换到另外一辆火车，比如站点换乘）
- 同一进程下不同线程间数据很易共享（A车厢换到B车厢很容易）
- 进程要比线程消耗更多的计算机资源（采用多列火车相比多个车厢更耗资源）
- 进程间不会相互影响，一个线程挂掉将导致整个进程挂掉（一列火车不会影响到另外一列火车，但是如果一列火车上中间的一节车厢着火了，将影响到所有车厢）
- 进程可以拓展到多机，进程最多适合多核（不同火车可以开在多个轨道上，同一火车的车厢不能在行进的不同的轨道上）
- 进程使用的内存地址可以上锁，即一个线程使用某些共享内存时，其他线程必须等它结束，才能使用这一块内存。（比如火车上的洗手间）－"互斥锁"
- 进程使用的内存地址可以限定使用量（比如火车上的餐厅，最多只允许多少人进入，如果满了需要在门口等，等有人出来了才能进去）－“信号量”


#  实验Executor.map方法

In [8]:
from time import sleep, strftime
from concurrent import futures

def display(*args):
    print(strftime('[%H:%M:%S]'), end = ' ')
    print(*args)

def loiter(n):
    msg = '{}loiter({}) : doing nothing for {}s'
    display(msg.format('\t'*n, n, n))
    sleep(n)
    msg = '{}loiter({}) : done'
    display(msg.format('\t'*n, n))
    return n * 10

def main():
    display('Script Starting.')
    executor = futures.ThreadPoolExecutor(max_workers=3)
    results = executor.map(loiter, range(5)) # 这一行表明，executor.map方法返回的结果（results）是生成器；不管有多少任务，也不管max_workers的值是多少，目前不会阻塞。
    display('results:', results)
    display('Waiting for individual results:')
    for i, result in enumerate(results):
        display('result {} : {}'.format(i, result))

main()


[21:36:12] Script Starting.
[21:36:12][21:36:12] 	loiter(1) : doing nothing for 1s
 loiter(0) : doing nothing for 0s
[21:36:12] loiter(0) : done
[21:36:12] 		loiter(2) : doing nothing for 2s
[21:36:12] results: <generator object Executor.map.<locals>.result_iterator at 0x7fd206b9db30>
[21:36:12] Waiting for individual results:
[21:36:12] [21:36:12]result 0 : 0
 			loiter(3) : doing nothing for 3s
[21:36:13] 	loiter(1) : done
[21:36:13] 				loiter(4) : doing nothing for 4s
[21:36:13] result 1 : 10
[21:36:14] 		loiter(2) : done
[21:36:14] result 2 : 20
[21:36:15] 			loiter(3) : done
[21:36:15] result 3 : 30
[21:36:17] 				loiter(4) : done
[21:36:17] result 4 : 40


for循环中的enumerate函数会隐式调用next(results)，这个函数又会在（内部）表示第一个任务（loiter(0)）的_f future上调用_f.result（　）方法。result方法会阻塞，直到future运行结束，因此这个循环每次迭代时都要等待下一个结果做好准备。